### This notebook contains the code used to evaluate the experiments and generate the LaTeX tables

In [231]:
import numpy as np
import pandas as pd
import eval_utils as eu
import matplotlib.pyplot as plt
import tensorflow as tf
import eval_utils as eu
import importlib
import pickle
import ranking_utils as ru
from utils.euler import one_step_euler
import signatory
import torch
import sigmmd

<module 'ranking_utils' from 'd:\\git\\deepSDE\\ranking_utils.py'>

In [2]:
synth1d_problems = ["ou", "cir", "sin", "sit"]
bio_problems = ["gfp"]
all_problems = synth1d_problems + bio_problems

## First - test direct coefficient reconstruction

In [128]:
ou_L = 2.0
ou_sigma = 2.0

cir_alpha = 2.0
cir_equilibrium = 4.0
cir_sigma = 2.0

sin_alpha = 15.0
sin_gamma = 2.0
sin_sigma = 1.0

sit_alpha = 6.0
sit_beta = 6.0
sit_gamma = 0.05
sit_sigma = 2.0

true_funcs = {
    "ou": (lambda x, t: -ou_L*x, lambda x, t: ou_sigma * np.ones_like(x)),
    "cir": (lambda x, t: -cir_alpha*(x-cir_equilibrium), lambda x, t: cir_sigma * (np.sqrt(x) * (x > 0.0))),
    "sin": (lambda x, t: +sin_alpha*(np.sin(x / (2*np.pi))) - sin_gamma * x, lambda x, t: sin_sigma * (1 + np.abs(x))),
    "sit": (lambda x, t: -sit_alpha * (np.sin(x)) + sit_beta * np.sin(t) - sit_gamma * x, lambda x, t: sit_sigma * np.ones_like(x))
}

true_a_funcs = {
    problem: a_func for (problem, (a_func, b_func)) in true_funcs.items()
}
true_b_funcs = {
    problem: b_func for (problem, (a_func, b_func)) in true_funcs.items()
}

In [129]:
train_sample_maxvals = {
    problem: np.amax(np.abs(np.load(f"./problems/{problem}_test/{problem}_train_samples.npy")).flatten()) for problem in all_problems
}
train_sample_maxvals

{'ou': 12.911348915685698,
 'cir': 18.662937416932802,
 'sin': 63.04767623291961,
 'sit': 27.361162601919897,
 'slv': 116.0866032509055,
 'gfp': 9.301496663593046}

In [29]:
def evaluate_model(model, grid, problem):
    scaled_grid = grid.copy()
    scaled_grid[:, 0] /= train_sample_maxvals[problem]
    
    pred = model.predict(scaled_grid)
    a_pred = pred[:, 0] * train_sample_maxvals[problem]
    b_pred = pred[:, 1] * train_sample_maxvals[problem]
    
    return a_pred, b_pred

In [13]:
test_samples = {
    problem: np.load(f"./problems/{problem}_test/{problem}_test_samples.npy") for problem in all_problems
}
tspans = {
    problem: np.load(f"./problems/{problem}_test/{problem}_tspan.npy") for problem in all_problems
}

In [14]:
methods = [
    "euler",
    "lt",
    "moment",
    "wasserstein",
    "corr",
    "euler_moment",
    "euler_wasserstein",
    "euler_corr",
    "lt_moment",
    "lt_wasserstein",
    "lt_corr",
    "euler_moment_corr",
    "euler_wasserstein_corr",
    "lt_moment_corr",
    "lt_wasserstein_corr"
]

In [112]:
N_SAMPLES = 1000

def sample_from_data(test_samples, tspan, rng, N):
    # Sample N random time points and pair them with N random test samples
    
    # Sample N random time points
    tpoint_idc = rng.choice(np.arange(tspan.shape[0]), size=N, replace=True)
    
    # Sample N random test samples
    sample_idc = rng.choice(np.arange(test_samples.shape[0]), size=N, replace=True)
    
    # Pair them together
    joint_grid = np.stack([(test_samples[sample_idx, tpoint_idx], tspan[tpoint_idx]) for sample_idx, tpoint_idx in zip(sample_idc, tpoint_idc)], axis=0)
    
    return joint_grid 

rng = np.random.default_rng(42)

data_grids = {}
for problem in all_problems:
    
    data_grid = sample_from_data(test_samples[problem], tspans[problem], rng, N_SAMPLES)
    data_grids[problem] = data_grid

### For assessing the L2 discrepancy between true and learned coefficients, we evaluate all models on a grid before comparing to the ground truth

In [113]:
data_grid_predictions = {}

for problem in synth1d_problems:
    for method in methods:
        print(f"Problem: {problem}, Method: {method}")
        model = tf.keras.models.load_model(f"./results/zipped_models/{problem}_{method}_model_16_64_64_16_100ep", compile=False)
        data_grid = data_grids[problem]
        a_pred, b_pred = evaluate_model(model, data_grid, problem)
        data_grid_predictions[(problem, method)] = (a_pred, b_pred)

Problem: ou, Method: euler
32/32 [==============================] - 0s 678us/step
Problem: ou, Method: lt
32/32 [==============================] - 0s 773us/step
Problem: ou, Method: moment
32/32 [==============================] - 0s 774us/step
Problem: ou, Method: wasserstein
32/32 [==============================] - 0s 903us/step
Problem: ou, Method: corr
32/32 [==============================] - 0s 742us/step
Problem: ou, Method: euler_moment
32/32 [==============================] - 0s 742us/step
Problem: ou, Method: euler_wasserstein
32/32 [==============================] - 0s 677us/step
Problem: ou, Method: euler_corr
32/32 [==============================] - 0s 677us/step
Problem: ou, Method: lt_moment
32/32 [==============================] - 0s 677us/step
Problem: ou, Method: lt_wasserstein
32/32 [==============================] - 0s 677us/step
Problem: ou, Method: lt_corr
32/32 [==============================] - 0s 742us/step
Problem: ou, Method: euler_moment_corr
32/32 [==========

In [114]:
with open("./results/data_grid_predictions_proper.pkl", "wb") as f:
    pickle.dump(data_grid_predictions, f)

In [137]:
drift_results = {}
diffusion_results = {}

for problem in synth1d_problems:  
    for method in methods:
        data_grid = data_grids[problem]
        
        true_a_values = true_a_funcs[problem](data_grid[:, 0], data_grid[:, 1])
        true_b_values = true_b_funcs[problem](data_grid[:, 0], data_grid[:, 1])
        
        pred_a_values, pred_b_values = data_grid_predictions[(problem, method)]
        
        drift_results[(problem, method)] = {
            "a_l2": np.linalg.norm(true_a_values - pred_a_values, 2) / len(true_a_values),
        }
        
        diffusion_results[(problem, method)] = {
            "b_l2": np.linalg.norm(true_b_values - pred_b_values, 2) / len(true_b_values),
        }      

In [138]:
ru.write_latex_table_content(synth1d_problems, drift_results, "a_l2_vals_proper.txt", cell="value")
ru.write_latex_table_content(synth1d_problems, ["lt_wasserstein_corr", "moment"], drift_results, "a_l2_vals_short.txt", cell="value")

In [140]:
ru.write_latex_table_content(synth1d_problems, diffusion_results, "b_l2_vals_proper.txt", cell="value")
ru.write_latex_table_content(synth1d_problems, ["lt_wasserstein_corr", "moment"], diffusion_results, "b_l2_vals_short.txt", cell="value")

### For assessing the capacity to generate samples, we simulate paths from the true and estimated dynamics using the same Brownian Motion sample paths before comparing

In [199]:
def one_step_euler_alternative(ab_fun, tspan, bms, y0, assert_positive=False):
    assert y0.shape[0] == bms.shape[0]

    res = np.zeros(shape=bms.shape)
    res[:, 0] = y0

    for i in range(1, bms.shape[1]):
        ab_vals = ab_fun(res[:, i-1], tspan[i-1])
        a_vals = ab_vals[:, 0]
        b_vals = ab_vals[:, 1]
        
        delta_t = tspan[i] - tspan[i-1]
        diff_add = delta_t * a_vals
        if assert_positive:
            diff_add[res[:, i-1] < 0.0] = 0.0
        res[:, i] = res[:, i-1] + diff_add + b_vals * (bms[:, i] - bms[:, i-1])

    return res

In [202]:
def eval_l2(problem, test_samples, tspan, test_bms, model, rng=None):
    if rng is None:
        rng = np.random.default_rng(42)

    initial_condition = test_samples[:, 0]
    
    
    true_samples = one_step_euler(
        a_fun = true_a_funcs[problem],
        b_fun = true_b_funcs[problem],
        bms=test_bms,
        tspan=tspan,
        y0=initial_condition
    )
    
    batch_model_ab = lambda x_vec, t: model.predict(np.stack([x_vec, t * np.ones(x_vec.shape)], axis=1),
                                                               verbose=False)

    initial_condition = test_samples[:, 0]
    
    scaled_initial_condition = initial_condition / train_sample_maxvals[problem]
    
    synth_samples = one_step_euler_alternative(batch_model_ab, tspan, test_bms, scaled_initial_condition)
    synth_samples = synth_samples * train_sample_maxvals[problem]
    
    return np.sum((true_samples - synth_samples) ** 2)
        

In [208]:
simulation_eval_l2_results = {}

In [209]:

for problem in synth1d_problems:
    for method in methods:

        test_samples = np.load(f"./problems/{problem}_test/{problem}_test_samples.npy")
        tspan = np.load(f"./problems/{problem}_test/{problem}_tspan.npy")
        gen_samples = np.load(f"./results/{problem}_test/{method}/generated_samples.npy")
        
        model = tf.keras.models.load_model(f"./results/zipped_models/{problem}_{method}_model_16_64_64_16_100ep", compile=False)
        test_bms = np.load(f"./problems/{problem}_test/{problem}_test_bms.npy")
        
        simulation_eval_l2_results[(problem, method)] = {
            "l2": eval_l2(problem, test_samples, tspan, test_bms, model) / (test_samples.shape[0] * test_samples.shape[1])
        }     
        print(f"Problem: {problem}, Method: {method} done")

Problem: ou, Method: euler done
Problem: ou, Method: lt done
Problem: ou, Method: moment done
Problem: ou, Method: wasserstein done
Problem: ou, Method: corr done
Problem: ou, Method: euler_moment done
Problem: ou, Method: euler_wasserstein done
Problem: ou, Method: euler_corr done
Problem: ou, Method: lt_moment done
Problem: ou, Method: lt_wasserstein done
Problem: ou, Method: lt_corr done
Problem: ou, Method: euler_moment_corr done
Problem: ou, Method: euler_wasserstein_corr done
Problem: ou, Method: lt_moment_corr done
Problem: ou, Method: lt_wasserstein_corr done
Problem: cir, Method: euler done
Problem: cir, Method: lt done
Problem: cir, Method: moment done
Problem: cir, Method: wasserstein done
Problem: cir, Method: corr done
Problem: cir, Method: euler_moment done
Problem: cir, Method: euler_wasserstein done
Problem: cir, Method: euler_corr done
Problem: cir, Method: lt_moment done
Problem: cir, Method: lt_wasserstein done
Problem: cir, Method: lt_corr done
Problem: cir, Method:

In [241]:
ru.write_latex_table_content(synth1d_problems, methods, simulation_eval_l2_results, "simulation_errors_l2.txt", cell="value")
ru.write_latex_table_content(synth1d_problems, ["lt_wasserstein_corr", "moment"], simulation_eval_l2_results, "simulation_errors_l2_short.txt", cell="value")

## Second - comparison without reference to the ground truth, by comparing testing samples vs. generated samples using the SigMMD metric

In [226]:
signature_mmds = {}

problems = ["ou", "cir", "sin", "sit", "gfp"]

for problem in problems:
    test_samples = np.load(f"./problems/{problem}_test/{problem}_test_samples.npy")
    tspan = np.load(f"./problems/{problem}_test/{problem}_tspan.npy")
    
    for method in methods:        
        gen_samples = np.load(f"./results/{problem}_test/{method}/generated_samples.npy")
        
        permuted_test_samples = np.expand_dims(np.transpose(test_samples), axis=2)
        permuted_gen_samples = np.expand_dims(np.transpose(gen_samples), axis=2)
        
        signature_mmds[(problem, method)] = sigmmd.compute_conventional_SigMMD(permuted_test_samples, permuted_gen_samples)
    
    uppercase_problem = problem.upper()
    sdegan_samples = np.load(f"./GANSDE_samples_6000/GANSDE_samples_6000/{uppercase_problem}_GANSDE_samples.npy")
    
    signature_mmds[(problem, "sdegan")] = sigmmd.compute_conventional_SigMMD(permuted_test_samples, np.expand_dims(np.transpose(sdegan_samples), axis=2))

In [239]:
ru.write_latex_table_content(synth1d_problems + ["gfp"], methods + ["sdegan"], {key: {"mmd": val} for key, val in signature_mmds.items()}, "sigmmd_errors.txt", cell="value", withsdegan=True)
ru.write_latex_table_content(synth1d_problems + ["gfp"], ["lt_wasserstein_corr", "moment", "sdegan"], {key: {"mmd": val} for key, val in signature_mmds.items()}, "sigmmd_errors_short.txt", cell="value", withsdegan=True)